In [15]:
from collections import deque
import re

In [1]:
inp = """Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A."""

In [59]:
with open('data/input_25.txt') as fh:
    inp = fh.read().strip()

In [54]:
def last_word(line):
    words = line.split()
    return re.sub('[^A-Za-z0-9]', '', words[-1])

In [60]:
class tmachine():
    def __init__(self, instructions=None):
        self.states = {}
        self.state = None
        self.tape = deque([0])
        self.pos = 0
        self.chksum_after = 0
        if instructions:
            self.load(instructions)
            
    def load(self, instructions):
        blocks = instructions.split('\n\n')
        init = blocks[0].split('\n')
        self.state = last_word(init[0])
        self.chksum_after = int(re.sub('[^0-9]', '', init[1]))
        
        for block in blocks[1:]:
            lines = block.split('\n')
            state = last_word(lines[0])
            self.states[state] = {}
            for tmp in (lines[1:5], lines[5:]):
                val = int(last_word(tmp[0]))
                self.states[state]['write_if_' + str(val)] = int(last_word(tmp[1]))
                self.states[state]['move_if_' + str(val)] = last_word(tmp[2])
                self.states[state]['next_state_if_' + str(val)] = last_word(tmp[3])
            
    def run(self):
        step = 0
        while True:
            step += 1
            val = self.tape[self.pos]
            self.tape[self.pos] = self.states[self.state]['write_if_' + str(val)]
            if self.states[self.state]['move_if_' + str(val)] == 'left':
                self.mv_left()
            else:
                self.mv_right()
            self.state = self.states[self.state]['next_state_if_' + str(val)]
            if step >= self.chksum_after:
                break
            #print(self.pos, self.tape)
        print('chksum: ', sum(self.tape))
            
    def mv_left(self):
        if self.pos == 0:
            self.tape.appendleft(0)
        else:
            self.pos -= 1
        return self.tape[self.pos]

    def mv_right(self):
        self.pos += 1
        if self.pos == len(self.tape):
            self.tape.append(0)
        return self.tape[self.pos]
    

    

In [65]:
tm = tmachine(inp)
tm.states

{'A': {'move_if_0': 'right',
  'move_if_1': 'left',
  'next_state_if_0': 'B',
  'next_state_if_1': 'E',
  'write_if_0': 1,
  'write_if_1': 0},
 'B': {'move_if_0': 'left',
  'move_if_1': 'right',
  'next_state_if_0': 'C',
  'next_state_if_1': 'A',
  'write_if_0': 1,
  'write_if_1': 0},
 'C': {'move_if_0': 'left',
  'move_if_1': 'right',
  'next_state_if_0': 'D',
  'next_state_if_1': 'C',
  'write_if_0': 1,
  'write_if_1': 0},
 'D': {'move_if_0': 'left',
  'move_if_1': 'left',
  'next_state_if_0': 'E',
  'next_state_if_1': 'F',
  'write_if_0': 1,
  'write_if_1': 0},
 'E': {'move_if_0': 'left',
  'move_if_1': 'left',
  'next_state_if_0': 'A',
  'next_state_if_1': 'C',
  'write_if_0': 1,
  'write_if_1': 1},
 'F': {'move_if_0': 'left',
  'move_if_1': 'right',
  'next_state_if_0': 'E',
  'next_state_if_1': 'A',
  'write_if_0': 1,
  'write_if_1': 1}}

In [66]:
%%time
tm.run()

chksum:  4387
CPU times: user 35.3 s, sys: 227 ms, total: 35.5 s
Wall time: 39 s
